## Aim

We want to solve a PDE of the form

$$
\partial_t \mathbf{Q} + \partial_{x_i} \mathbf{F}_i = NC_i \partial_{x_i} \mathbf{Q} + \mathbf{S}
$$

where $\mathbf{F}$ is a flux, $NC$ is a non-conservative matrix, $\mathbf{S}$ is a source term and dimension $i=[1]$ for 1d, $i=[1,2]$ for 2d and $i = [1, 2,3]$ for 3d.



## Imports

In [9]:
#| code-fold: true
#| code-summary: "Load packages"
#| output: false

import os
import numpy as np
import pyvista as pv
import matplotlib.pyplot as plt
from scipy.interpolate import griddata
from copy import deepcopy
import seaborn 
seaborn.set_context('talk')
from IPython.display import Math
import sympy as sym

main_dir = os.getenv("SMS")
import pytest
from types import SimpleNamespace

from library.model.model import *
from library.pysolver.solver import *
from library.pysolver.solver import jax_fvm_unsteady_semidiscrete as fvm_unsteady
import library.model.initial_conditions as IC
import library.model.boundary_conditions as BC
from library.pysolver.ode import RK1
import library.misc.io as io
from library.pysolver.reconstruction import GradientMesh
import library.mesh.mesh as petscMesh
import library.postprocessing.postprocessing as postprocessing


## 1d Model

### Model definition

In [178]:

class ShallowMomentsSediment(Model):
    """
    Shallow Moments Sediment 1d

    :gui: 
    - tab: model
    - requires: [ 'mesh.dimension': 1 ]

    """
    def __init__(
        self,
        boundary_conditions,
        initial_conditions,
        aux_initial_conditions=IC.Constant(),
        dimension=1,
        fields=3,
        aux_fields=0,
        parameters = {},
        parameters_default={"g": 1.0, "ex": 0.0, "ez": 1.0},
        settings={},
        settings_default={"topography": False, "friction": []},
        basis=Basis()
    ):
        self.basis = basis
        self.variables = register_sympy_attribute(fields, "q")
        self.n_fields = self.variables.length()
        self.levels = self.n_fields - 3
        self.basis.compute_matrices(self.levels)
        if type(fields) == type([]):
            fields_smm = fields[:-1]
        elif type(fields) == int:
            fields_smm = fields-1
        else:
            assert False
        self.smm = ShallowMoments(
            dimension=dimension,
            fields=fields_smm,
            aux_fields=aux_fields,
            parameters=parameters,
            parameters_default = parameters_default,
            boundary_conditions=boundary_conditions,
            initial_conditions=initial_conditions,
            aux_initial_conditions=aux_initial_conditions,
            settings={**settings_default, **settings},
        )
        super().__init__(
            dimension=dimension,
            fields=fields,
            aux_fields=aux_fields,
            parameters=parameters,
            parameters_default = parameters_default,
            boundary_conditions=boundary_conditions,
            initial_conditions=initial_conditions,
            aux_initial_conditions=aux_initial_conditions,
            settings={**settings_default, **settings},
        )

    def flux(self):
        flux = Matrix([0 for i in range(self.n_fields)])
        # smm flux
        flux[:-1,0] = self.smm.sympy_flux[0]

        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        a = [_ha/h for _ha in ha]
        p = self.parameters
        # bedload equation
        ub = a[0]
        for i in range(self.levels):
            ub += a[i+1]
        tau = p.epsilon * p.rho * sym.Abs(ub) * ub
        theta = (sym.Abs(tau)*p.d_s**2)/(p.g * (p.rho_s - p.rho) * d_s**3)
        def pos(x):
            return sym.Piecewise((0, x<0), (x, True))
        def sign(x):
            return sym.sign(x)
        # flux[-1, 0]  = p.Q * (sign(tau)*8/(1-p.phi) * pos(theta - p.theta_c)**(3/2) ) 
        return [flux]

    def nonconservative_matrix(self):
        nc = Matrix([[0 for i in range(self.n_fields)] for j in range(self.n_fields)])
        nc[:-1, :-1] = self.smm.sympy_nonconservative_matrix[0]
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        a = [_ha / h for _ha in ha]
        p = self.parameters
        um = ha[0]/h
        ub = a[0]
        for i in range(self.levels):
            ub += a[i+1]
        tau = p.epsilon * p.rho * sym.Abs(ub) * ub
        theta = (sym.Abs(tau)*p.d_s**2)/(p.g * (p.rho_s - p.rho) * d_s**3)
        def pos(x):
            return sym.Piecewise((0, x<0), (x, True))
        def sign(x):
            return sym.sign(x)
        delta_q = (24 * p.Q)/(1-p.phi) * sign(tau) * p.epsilon / (p.g * (1/p.r - 1) * p.d_s) * pos(theta - p.theta_c)**(1/2) * ub / h
        # delta_q = (24 * p.Q)/(1-p.phi) * sign(ub) * p.epsilon / (p.g * (1/p.r - 1) * p.d_s) * pos(theta - p.theta_c)**(1/2) * ub / h
        # delta_q = (24 * p.Q)/(1-p.phi) * sign(ub) * p.epsilon / (p.g * (1/p.r - 1) * p.d_s) 
        # delta_q = (24 * p.Q)/(1-p.phi) * sign(ub) 
        # delta_q = 10**(-6) * sign(ub)
        # delta_q = 0
        nc[-1, 0] = - ub * delta_q
        for l in range(self.levels+1):
            nc[-1, l+1] = delta_q
        return [nc]

    def eigenvalues(self):
        A = self.sympy_normal[0] * self.sympy_quasilinear_matrix[0]
        for d in range(1, self.dimension):
            A += self.sympy_normal[d] * self.sympy_quasilinear_matrix[d]
        if self.levels > 1:
            alpha_erase = self.variables[2:-1]
            for alpha_i in alpha_erase:
                A = A.subs(alpha_i, 0)
        for i in range(self.n_fields):
            A[-1, i] = 0
        return eigenvalue_dict_to_matrix(A.eigenvals())

    def source(self):
        out = Matrix([0 for i in range(self.n_fields)])
        if self.settings.topography:
            out += self.topography()
        if self.settings.friction:
            for friction_model in self.settings.friction:
                out += getattr(self, friction_model)()
        return out


    def newtonian(self):
        """
        :gui:
            - requires_parameter: ('nu', 0.0)
        """
        assert "nu" in vars(self.parameters)
        out = Matrix([0 for i in range(self.n_fields)])
        out[:-1] = self.smm.newtonian()
        return out

    def chezy(self):
        """
        :gui:
            - requires_parameter: ('C', 1000.0)
        """
        assert "C" in vars(self.parameters)
        out = Matrix([0 for i in range(self.n_fields)])
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        p = self.parameters
        tmp = 0
        for i in range(1+self.levels):
            for j in range(1+self.levels):
                tmp += ha[i] * ha[j] / h / h
        sqrt = sympy.sqrt(tmp)
        for k in range(1+self.levels):
            for l in range(1+self.levels):
                out[1+k] += -1./(p.C**2 * self.basis.M[k,k]) * ha[l] * sqrt / h 
        return out

### Mesh and model construction

In [179]:
#| code-fold: true
#| code-summary: "Initialize model and mesh"
mesh = petscMesh.Mesh.create_1d((-5, 5), 100)

bcs = BC.BoundaryConditions(
    [
        BC.Extrapolation(physical_tag='left'),
        BC.Extrapolation(physical_tag="right"),
    ]
)

level = 0
def custom_ic(x):
    Q = np.zeros(1 + level+1+1, dtype=float)
    Q[0] = np.where(x[0] < 0, 1., 0.05)
    return Q

ic = IC.UserFunction(custom_ic)


d_s = 3.9*10**(-3)
rho = 1000.
rho_s = 1580.
r = rho/rho_s
g = 9.81
phi = 0.47
# n = 1.
# d_50 = 1.
# epsilon = (g * n**2)/((7/6)**2 * d_50**(1/3))
epsilon = 0.0324
# Q = d_s * sym.sqrt(g * (1-r-1) * d_s)
Q = 0.
theta_c = 0.047


fields = ['h'] + [f'hu_{l}' for l in range(level+1)] + ['b']
model = ShallowMomentsSediment(
    boundary_conditions=bcs,
    initial_conditions=ic,
    fields=fields,
    settings={"friction": []},
    parameters = {"C": 1.0, "g": g, "ez": 1.0, "d_s":d_s, "rho":rho, "rho_s":rho_s, "r": r, "phi": phi, "n":n, "d_50":d_50, "epsilon": epsilon, "Q": Q, "theta_c":theta_c}
)


### Display model

In [180]:

#| code-fold: true
#| code-summary: "Display model"
display(Math(r'\large{' + 'Flux \, in \, x' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_flux[0])) + '}'))
display(Math(r'\large{' + 'Nonconservative \, matrix \, in \, x' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_nonconservative_matrix[0])) + '}'))
display(Math(r'\large{' + 'Eigenvalues' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_eigenvalues)) + '}'))
display(Math(r'\large{' + 'Source' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_source)) + '}'))

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

### Simulation

In [181]:
#| code-fold: false
#| code-summary: "Simulation"
#| output: false

settings = Settings(
    reconstruction=recon.constant,
    num_flux=flux.Zero(),
    nc_flux=nonconservative_flux.segmentpath(3),
    compute_dt=timestepping.adaptive(CFL=.1),
    time_end=1.,
    output_snapshots=100,
    output_clean_dir=True,
    output_dir="outputs/demo/swe",
)


solver_price_c(mesh, model, settings)


1 0.0031927542840705053 0.0031927542840705053-------------------- 
2 0.006385508568141011 0.0031927542840705053
3 0.009578262852211515 0.0031927542840705053
4 0.012771017136282021 0.0031927542840705053
5 0.01596253205024433 0.0031915149139623094
6 0.019151617924397152 0.0031890858741528217
7 0.022337357594043438 0.003185739669646286
8 0.02551906025628929 0.0031817026622458534
9 0.02869620265834085 0.0031771424020515587
10 0.03186838636192965 0.0031721837035887995
11 0.03503530814554625 0.0031669217836165975
12 0.03819673900186779 0.0031614308563215443
13 0.041352508709364515 0.0031557697074967267
14 0.04450249415015856 0.0031499854407940468
15 0.04764661025508095 0.0031441161049223826
16 0.050784802861803466 0.003138192606722518
17 0.05391704300830166 0.003132240146498199
18 0.05704332232800679 0.003126279319705123
19 0.0601636493052408 0.0031203269772340117
20 0.06327804621133974 0.003114396906098949
21 0.06638654658495842 0.0031085003736186774
22 0.069489193151066 0.00310264656610757

<lambdifygenerated-364>:5: RuntimeWarning: invalid value encountered in sqrt
  return (array([[1.0e-20*h, 1.0e-20*h + 1, 1.0e-20*h], [ez*g*h - 1.0*hu_0**2/h**2, 2.0*hu_0/h, 1.0e-20*h], [24*Q*epsilon*hu_0**2*select([less(16858005.0236855*d_s**2*abs(epsilon*hu_0*rho*abs(hu_0/h)/h)/(g*(-rho + rho_s)) - theta_c, 0),True], [0,4105.85009756634*sqrt(d_s**2*abs(epsilon*hu_0*rho*abs(hu_0/h)/h)/(g*(-rho + rho_s)) - 5.9319e-8*theta_c)], default=nan)*sign(epsilon*hu_0*rho*abs(hu_0/h)/h)/(d_s*g*h**3*(-1 + r**(-1.0))*(1 - phi)), -24*Q*epsilon*hu_0*select([less(16858005.0236855*d_s**2*abs(epsilon*hu_0*rho*abs(hu_0/h)/h)/(g*(-rho + rho_s)) - theta_c, 0),True], [0,4105.85009756634*sqrt(d_s**2*abs(epsilon*hu_0*rho*abs(hu_0/h)/h)/(g*(-rho + rho_s)) - 5.9319e-8*theta_c)], default=nan)*sign(epsilon*hu_0*rho*abs(hu_0/h)/h)/(d_s*g*h**2*(-1 + r**(-1.0))*(1 - phi)), 1.0e-20*h]]))


71 0.21549179027733176 0.0028817714626257636
72 0.2183701736654941 0.0028783833881623356
73 0.22124522310281078 0.002875049437316676
74 0.2241169914215463 0.002871768318735522
75 0.22698553020423373 0.002868538782687419
76 0.22985088982361424 0.0028653596193804988
77 0.23271311948097403 0.002862229657359788
78 0.235572267242954 0.0028591477619799538
79 0.2384283800769036 0.0028561128339495948
80 0.24128150388484698 0.0028531238079434014
81 0.24413168353612566 0.0028501796512786703
82 0.24697896289877852 0.002847279362652869
83 0.2498233848697176 0.0028444219709391066
84 0.25266499140375415 0.0028416065340365294
85 0.255503823541527 0.0028388321377728434
86 0.2583399214363833 0.002836097894856287
87 0.2611733243802578 0.002833402943874539
88 0.264003895746762 0.0028305713665041413
89 0.2668315008633889 0.0028276051166269646
90 0.2696561867398941 0.0028246858765052165
91 0.2724779992641398 0.0028218125242456616
92 0.27529698323910545 0.0028189839749656878
93 0.2781131824183112 0.00281619

Settings(name='Simulation', parameters={}, reconstruction=<function constant at 0x7fbecaada8b0>, reconstruction_edge=<function constant_edge at 0x7fbecaada940>, num_flux=<function Zero.<locals>.flux at 0x7fbe6d3a2670>, nc_flux=<function segmentpath.<locals>.nc_flux_quasilinear at 0x7fbe6d71a5e0>, compute_dt=<function adaptive.<locals>.compute_dt at 0x7fbe6d4ac0d0>, time_end=1.0, truncate_last_time_step=True, output_snapshots=100, output_write_all=False, output_dir='outputs/demo/swe', output_clean_dir=True, solver_code_base='python', callbacks=[], debug=False, profiling=False)

### Postprocessing

In [ ]:

#| code-fold: true
#| code-summary: "Postprocessing"
filepath = os.path.join(settings.output_dir, 'Simulation.h5')
X, Q, Qaux, T = io.load_timeline_of_fields_from_hdf5(filepath)
# remove the boundary points
Q = Q[:, :, :-2]
X = X[:-2]

In [ ]:
#| code-fold: false
#| code-summary: "Plot with Matplotlib"
fig, ax = plt.subplots(1, 2, figsize=(12, 8))
i_time = 0
ax[0].plot(X, Q[i_time, 0, :], label='rho')
# ax[0].plot(X, Q[i_time, 1, :], label='rho u')
# ax[0].plot(X, Q[i_time, 2, :], label='E')
ax[0].set_title(f'Time: {T[i_time]}')

i_time = -1
ax[1].plot(X, Q[i_time, 0, :], label='rho')
# ax[1].plot(X, Q[i_time, 1, :], label='rho u')
# ax[1].plot(X, Q[i_time, 2, :], label='E')
ax[1].set_title(f'Time: {np.round(T[i_time], 1)}')
plt.legend()

## 2d Model

### Model

In [ ]:

class ShallowMomentsSediment2d(Model):
    def __init__(
        self,
        boundary_conditions,
        initial_conditions,
        dimension=2,
        fields=3,
        aux_fields=0,
        parameters = {},
        parameters_default={"g": 1.0, "ex": 0.0, "ey": 0.0, "ez": 1.0},
        settings={},
        settings_default={"topography": False, "friction": []},
        basis=Basis()
    ):
        self.basis = basis
        self.variables = register_sympy_attribute(fields, "q")
        self.n_fields = self.variables.length()
        self.levels = int((self.n_fields - 1)/2)-1
        self.basis.compute_matrices(self.levels)
        super().__init__(
            dimension=dimension,
            fields=fields,
            aux_fields=aux_fields,
            parameters=parameters,
            parameters_default = parameters_default,
            boundary_conditions=boundary_conditions,
            initial_conditions=initial_conditions,
            settings={**settings_default, **settings},
        )

    def flux(self):
        offset = self.levels+1
        flux_x = Matrix([0 for i in range(self.n_fields)])
        flux_y = Matrix([0 for i in range(self.n_fields)])
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        hb = self.variables[1+self.levels+1:1+2*(self.levels+1)]
        p = self.parameters
        flux_x[0] = ha[0]
        flux_x[1] = p.g * p.ez * h * h / 2
        for k in range(self.levels+1):
            for i in range(self.levels+1):
                for j in range(self.levels+1):
                    # TODO avoid devision by zero 
                    flux_x[k+1] += ha[i] * ha[j] / h * self.basis.A[k, i, j] / self.basis.M[ k, k ]
        for k in range(self.levels+1):
            for i in range(self.levels+1):
                for j in range(self.levels+1):
                    # TODO avoid devision by zero 
                    flux_x[k+1+offset] += hb[i] * ha[j] / h * self.basis.A[k, i, j] / self.basis.M[ k, k ]

        flux_y[0] = hb[0]
        flux_y[1+offset] = p.g * p.ez * h * h / 2
        for k in range(self.levels+1):
            for i in range(self.levels+1):
                for j in range(self.levels+1):
                    # TODO avoid devision by zero 
                    flux_y[k+1] += hb[i] * ha[j] / h * self.basis.A[k, i, j] / self.basis.M[ k, k ]
        for k in range(self.levels+1):
            for i in range(self.levels+1):
                for j in range(self.levels+1):
                    # TODO avoid devision by zero 
                    flux_y[k+1+offset] += hb[i] * hb[j] / h * self.basis.A[k, i, j] / self.basis.M[ k, k ]
        return [flux_x, flux_y]

    def nonconservative_matrix(self):
        offset = self.levels+1
        nc_x = Matrix([[0 for i in range(self.n_fields)] for j in range(self.n_fields)])
        nc_y = Matrix([[0 for i in range(self.n_fields)] for j in range(self.n_fields)])
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        hb = self.variables[1+offset:1+offset+self.levels+1]
        p = self.parameters
        um = ha[0]/h
        vm = hb[0]/h
        for k in range(1, self.levels+1):
            nc_x[k+1, k+1] += um
            nc_y[k+1, k+1+offset] += um
        for k in range(self.levels+1):
            for i in range(1, self.levels+1):
                for j in range(1, self.levels+1):
                    nc_x[k+1, i+1] -= ha[j]/h*self.basis.B[k, i, j]/self.basis.M[k, k]
                    nc_y[k+1, i+1+offset] -= ha[j]/h*self.basis.B[k, i, j]/self.basis.M[k, k]

        for k in range(1, self.levels+1):
            nc_x[k+1+offset, k+1] += vm
            nc_y[k+1+offset, k+1+offset] += vm
        for k in range(self.levels+1):
            for i in range(1, self.levels+1):
                for j in range(1, self.levels+1):
                    nc_x[k+1+offset, i+1] -= hb[j]/h*self.basis.B[k, i, j]/self.basis.M[k, k]
                    nc_y[k+1+offset, i+1+offset] -= hb[j]/h*self.basis.B[k, i, j]/self.basis.M[k, k]
        return [nc_x, nc_y]

    def eigenvalues(self):
        # we delete heigher order moments (level >= 2) for analytical eigenvalues
        offset = self.levels+1
        A = self.sympy_normal[0] * self.sympy_quasilinear_matrix[0]
        for d in range(1, self.dimension):
            A += self.sympy_normal[d] * self.sympy_quasilinear_matrix[d]
        alpha_erase = self.variables[2:2+self.levels]
        beta_erase = self.variables[2+offset : 2+offset+self.levels]
        for alpha_i in alpha_erase:
            A = A.subs(alpha_i, 0)
        for beta_i in beta_erase:        
            A = A.subs(beta_i, 0)
        return eigenvalue_dict_to_matrix(A.eigenvals())

    def source(self):
        out = Matrix([0 for i in range(self.n_fields)])
        if self.settings.topography:
            out += self.topography()
        if self.settings.friction:
            for friction_model in self.settings.friction:
                out += getattr(self, friction_model)()
        return out

    def topography(self):
        assert "dhdx" in vars(self.aux_variables)
        assert "dhdy" in vars(self.aux_variables)
        offset = self.levels+1
        out = Matrix([0 for i in range(self.n_fields)])
        h = self.variables[0]
        p = self.parameters
        dhdx = self.aux_variables.dhdx
        dhdy = self.aux_variables.dhdy
        out[1] = h * p.g * (p.ex - p.ez * dhdx)
        out[1+offset] = h * p.g * (p.ey - p.ez * dhdy)
        return out


    def newtonian(self):
        assert "nu" in vars(self.parameters)
        out = Matrix([0 for i in range(self.n_fields)])
        offset = self.levels+1
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        hb = self.variables[1+offset:1+self.levels+1+offset]
        p = self.parameters
        for k in range(1+self.levels):
            for i in range(1+self.levels):
                out[1+k] += -p.nu/h * ha[i]  / h * self.basis.D[i, k]/ self.basis.M[k, k]
                out[1+k+offset] += -p.nu/h * hb[i]  / h * self.basis.D[i, k]/ self.basis.M[k, k]
        return out


    def slip(self):
        assert "lamda" in vars(self.parameters)
        assert "rho" in vars(self.parameters)
        out = Matrix([0 for i in range(self.n_fields)])
        offset = self.levels+1
        h = self.variables[0]
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        hb = self.variables[1+offset:1+self.levels+1+offset]
        p = self.parameters
        for k in range(1+self.levels):
            for i in range(1+self.levels):
                out[1+k] += -1./p.lamda/p.rho * ha[i]  / h / self.basis.M[k, k]
                out[1+k+offset] += -1./p.lamda/p.rho * hb[i]  / h / self.basis.M[k, k]
        return out

    def chezy(self):
        assert "C" in vars(self.parameters)
        out = Matrix([0 for i in range(self.n_fields)])
        offset = self.levels+1
        h = self.variables[0]
        ha = self.variables[1:1+self.levels+1]
        hb = self.variables[1+offset:1+self.levels+1+offset]
        p = self.parameters
        tmp = 0
        for i in range(1+self.levels):
            for j in range(1+self.levels):
                tmp += ha[i] * ha[j] / h / h + hb[i] * hb[j] / h / h
        sqrt = sympy.sqrt(tmp)
        for k in range(1+self.levels):
            for l in range(1+self.levels):
                out[1+k] += -1./(p.C**2 * self.basis.M[k,k]) * ha[l] * sqrt / h 
                out[1+k+offset] += -1./(p.C**2 * self.basis.M[k,k]) * hb[l] * sqrt / h 
        return out

### Mesh

In [ ]:
#| code-fold: false
#| code-summary: "Load Gmsh mesh"

# mesh = petscMesh.Mesh.from_gmsh(os.path.join(main_dir, "meshes/quad_2d/mesh_coarse.msh"))
mesh = petscMesh.Mesh.from_gmsh(os.path.join(main_dir, "meshes/quad_2d/mesh_fine.msh"))
# mesh = petscMesh.Mesh.from_gmsh(os.path.join(main_dir, "meshes/quad_2d/mesh_finest.msh"))
# mesh = petscMesh.Mesh.from_gmsh(os.path.join(main_dir, "meshes/triangle_2d/mesh_finest.msh"))
print(f"physical tags in gmsh file: {mesh.boundary_conditions_sorted_names}")


### Model

In [ ]:
#| code-summary: "Model"
#| code-fold: false

bcs = BC.BoundaryConditions(
    [
        BC.Periodic(physical_tag='left', periodic_to_physical_tag='right'),
        BC.Periodic(physical_tag="right", periodic_to_physical_tag='left'),
        BC.Periodic(physical_tag='top', periodic_to_physical_tag='bottom'),
        BC.Periodic(physical_tag="bottom", periodic_to_physical_tag='top'),
    ]
)


def custom_ic(x):
    Q = np.zeros(3, dtype=float)
    Q[0] = np.where(x[0]**2 + x[1]**2 < 0.1, 2., 1.)
    return Q

ic = IC.UserFunction(custom_ic)


model = ShallowMomentsSediment2d(
    boundary_conditions=bcs,
    initial_conditions=ic,
    fields=['h', 'hu', 'hv'],
    settings={"friction": ["chezy"]},
    parameters = {"C": 1.0, "g": 9.81, "ez": 1.0}
)




### Automatic tex generation

In [ ]:
#| code-summary: "Display Model"
#| code-fold: true
display(Math(r'\large{' + 'Flux \, in \, x' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_flux[0])) + '}'))
display(Math(r'\large{' + 'Flux \, in \, y' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_flux[1])) + '}'))
display(Math(r'\large{' + 'Nonconservative \, matrix \, in \, x' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_nonconservative_matrix[0])) + '}'))
display(Math(r'\large{' + 'Nonconservative \, matrix \, in \, y' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_nonconservative_matrix[1])) + '}'))
display(Math(r'\large{' + 'Quasilinear \, matrix \, in \, x' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_quasilinear_matrix[0])) + '}'))
display(Math(r'\large{' + 'Quasilinear \, matrix \, in \, y' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_quasilinear_matrix[1])) + '}'))
display(Math(r'\large{' + 'Eigenvalues' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_eigenvalues)) + '}'))
display(Math(r'\large{' + 'Source' + '}'))
display(Math(r'\large{' + sympy.latex(sympy.simplify(model.sympy_source)) + '}'))

### Simulation

In [ ]:
#| code-fold: false
#| code-summary: "Simulation"
#| output: false

settings = Settings(
    reconstruction=recon.constant,
    num_flux=flux.Zero(),
    nc_flux=nonconservative_flux.segmentpath(3),
    compute_dt=timestepping.adaptive(CFL=.45),
    time_end=1.,
    output_snapshots=100,
    output_clean_dir=True,
    output_dir="outputs/demo/swe",
)


solver_price_c(mesh, model, settings)


### Postprocessing

In [ ]:
#| code-fold: true
#| code-summary: "Postprocessing"
io.generate_vtk(os.path.join(os.path.join(main_dir, settings.output_dir), f'{settings.name}.h5'))
out_0 = pv.read(os.path.join(os.path.join(main_dir, settings.output_dir), 'out.0.vtk'))
out_10 = pv.read(os.path.join(os.path.join(main_dir, settings.output_dir), 'out.10.vtk'))
out_98 = pv.read(os.path.join(os.path.join(main_dir, settings.output_dir), 'out.98.vtk'))
field_names = out_0.cell_data.keys()
print(f'Field names: {field_names}')

In [ ]:
#| code-fold: true
#| code-summary: "Plot VTK"
p = pv.Plotter(shape=(1,3), notebook=True)

p.subplot(0, 0)
p.add_mesh(out_0, scalars='0', show_edges=False, scalar_bar_args={'title': 'h(t=0)'})
p.enable_parallel_projection()
p.enable_image_style()
p.view_xy()

p.subplot(0, 1)
p.add_mesh(out_10, scalars='0', show_edges=False, scalar_bar_args={'title': 'h(t=0.1)'})
p.enable_parallel_projection()
p.enable_image_style()
p.view_xy()

p.subplot(0, 2)
p.add_mesh(out_98, scalars='0', show_edges=False, scalar_bar_args={'title': 'h(t=1)'})
p.enable_parallel_projection()
p.enable_image_style()
p.view_xy()

p.show(jupyter_backend='static')

In [ ]:
#| code-fold: true
#| code-summary: "Postprocessing"
filepath = os.path.join(settings.output_dir, 'Simulation.h5')
X, Q, Qaux, T = io.load_timeline_of_fields_from_hdf5(filepath)
# remove the boundary points
Q = Q[:, :, :-2]
X = X[:-2]

### Postprocessing

In [ ]:
#| code-fold: false
#| code-summary: "Plot with Matplotlib"
fig, ax = plt.subplots(1, 2, figsize=(12, 8))
i_time = 0
ax[0].plot(X, Q[i_time, 0, :], label='rho')
ax[0].plot(X, Q[i_time, 1, :], label='rho u')
ax[0].plot(X, Q[i_time, 2, :], label='E')
ax[0].set_title(f'Time: {T[i_time]}')

i_time = -1
ax[1].plot(X, Q[i_time, 0, :], label='rho')
ax[1].plot(X, Q[i_time, 1, :], label='rho u')
ax[1].plot(X, Q[i_time, 2, :], label='E')
ax[1].set_title(f'Time: {np.round(T[i_time], 1)}')
plt.legend()